In [ ]:
from Capsule.Layer.CapsuleLayer import CapsuleRouting
import torch
routing = CapsuleRouting(mode='em')

v = torch.rand(2, 8, 10, 16, 1, 1)
a = torch.rand(2, 8, 1, 1)
v, a = routing(v, a)
print(a)

In [1]:
from Capsule.Layer.CapsuleLayer import AdaptiveCapsuleHead
import torch

model = AdaptiveCapsuleHead(32, 10, 4, 1, False, *["dynamic", 10, 1.5])
v = torch.rand(2, 32, 5, 5)

out, v = model(v, get_capsules=True)
print(v.shape)
print(v)

torch.Size([2, 10, 5, 5])
tensor([[[[0.0855, 0.0830, 0.2313, 0.0757, 0.0870],
          [0.0736, 0.0749, 0.2242, 0.0802, 0.2310],
          [0.0898, 0.0750, 0.0866, 0.0756, 0.0748],
          [0.0748, 0.0754, 0.0745, 0.2302, 0.2013],
          [0.0746, 0.0854, 0.0788, 0.0756, 0.0854]],

         [[0.0855, 0.2073, 0.0854, 0.0757, 0.0878],
          [0.1101, 0.0749, 0.0862, 0.1375, 0.0854],
          [0.0921, 0.1995, 0.0897, 0.1948, 0.0748],
          [0.0748, 0.1996, 0.0745, 0.0855, 0.0747],
          [0.2018, 0.0854, 0.0790, 0.0756, 0.2314]],

         [[0.0855, 0.0869, 0.0854, 0.0757, 0.0880],
          [0.1924, 0.0749, 0.0862, 0.0825, 0.0854],
          [0.0908, 0.0750, 0.0886, 0.0756, 0.2007],
          [0.0748, 0.0754, 0.0745, 0.0855, 0.0747],
          [0.0746, 0.0854, 0.0788, 0.0756, 0.0854]],

         [[0.0855, 0.0837, 0.0854, 0.0757, 0.0881],
          [0.0736, 0.0749, 0.0862, 0.0809, 0.0854],
          [0.0909, 0.0750, 0.0874, 0.0756, 0.0748],
          [0.0748, 0.0754, 0.074

/home/vips/anaconda3/envs/Capsule/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
import yaml
import torch
from Capsule.Layer.Classifier import CapsuleWrappingClassifier
with open('config/new-config.yml', 'r') as stream:
    PARAMS = yaml.safe_load(stream)
    print(PARAMS)
model = CapsuleWrappingClassifier(model_configs=PARAMS['architect_settings'])

img = torch.rand(2, 3, 224, 224)

out = model(img)
print(out.shape)
print(out)

{'logger': {'project': 'kaori/Capsule-Network', 'api_key': 'YOUR API KEY', 'tags': 'None'}, 'task': 'classification', 'architect_settings': {'name': 'classifier-test', 'backbone': {'name': 'swin-s', 'is_full': False, 'is_pretrained': True, 'is_freeze': False}, 'n_cls': 10, 'is_caps': True, 'caps': {'mode': 1, 'cap_dims': 4, 'routing': {'type': 'dynamic', 'params': [3, 1.5]}}}, 'dataset_settings': {'name': 'CIFAR10', 'path': 'data/', 'img_size': 224}, 'training_settings': {'gpu_ids': [0], 'n_gpu': 1, 'loss': 'nll', 'ckpt_path': './mode/test', 'n_epoch': 5, 'n_batch': 64, 'num_workers': 0, 'optimizer': 'adam', 'lr_scheduler': 'step', 'early_stopping': False, 'lr': 0.0001, 'lr_step': 10, 'lr_decay': 0.8, 'momentum': 0.9, 'weight_decay': 0.005}}
CapsuleWrappingClassifier(
  (preprocess): ImageClassification(
      crop_size=[224]
      resize_size=[232]
      mean=[0.485, 0.456, 0.406]
      std=[0.229, 0.224, 0.225]
      interpolation=InterpolationMode.BICUBIC
  )
  (backbone): SwinTrans

In [18]:
from torchvision.models import densenet121 as Model

model = Model()
print(model)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu